# Requirements

In [113]:
!pip install javalang

In [20]:
import os
import hashlib
import ast       #Library to produce AST and analyze it
import astor     #Library to produce the actual code of an AST
import pandas as pd
#import javalang
#from javalang.tree import *

In [10]:
hash_dict = {}
filesname = []
file_directories = []

In [11]:
"""
In this part we make a list of the files' name and their own directory in the given directory.
A dictionary is also produced having a hash value of the files name as the key and their actual name as the value.
"""

directory = "/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        filesname.append(filename)
        file_directories.append(f)

In [13]:
file_directories

['/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/oauth.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/timeline.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/basetest_project_root.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/repository.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/test_feeds.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/taskd.py',
 '/home/amirmahdi/projects/Adaptive-Logging-system-git/projects-to-investigate/allura-master/selected-files-to-investigate/__init__.py',
 '/home/amirmahdi/projects/Ada

# Extracting function blocks from source code

In [12]:
"""
This function accepts the source code as a string. It first parse it to AST, then iterate through AST and selects 
nodes that are representing function definition (FunctionDef)
"""

def extract_functions(source_code):
    tree = ast.parse(source_code)
    functions = []
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef):
            functions.append(node)
    return functions

In [17]:
"""
Creating the func_of_files list which each of its elements it is a list of function nodes of a file in the
directory.
These nodes (subtree of the AST) could be used to creat the actual functions in string by using following code:

source_code = astor.to_source(node)

Also, in this an ID is assigned to each function (based on its name) by using hashlib, then this ID is used as 
the key value of the "hash_dict" which holds a list containing function's name and its directory as the 
dictionary's value.
"""

func_nodes_of_files = []
for file_dir in file_directories:
    with open(file_dir, "r") as file:
        source_code = file.read()
    functions = extract_functions(source_code)
    for function in functions:
        hash_dict[ hashlib.sha1(str.encode(function.name)).hexdigest() ] = [file_dir.split("/")[-1], file_dir] 
    func_nodes_of_files.extend(functions)

In [18]:
func_nodes_of_files[0].name

'dummy_oauths'

In [19]:
source = astor.to_source(func_nodes_of_files[0])
print(source)

def dummy_oauths():
    from allura.controllers.rest import Oauth1Validator
    dummy_cons_tok = OAuthConsumerToken(api_key=Oauth1Validator().
        dummy_client, name='dummy client, for oauthlib implementation',
        user_id=None)
    session(dummy_cons_tok).flush(dummy_cons_tok)
    dummy_req_tok = OAuthRequestToken(api_key=Oauth1Validator().
        dummy_request_token, user_id=None, validation_pin='dummy-pin')
    session(dummy_req_tok).flush(dummy_req_tok)
    dummy_access_tok = OAuthAccessToken(api_key=Oauth1Validator().
        dummy_access_token, user_id=None)
    session(dummy_access_tok).flush(dummy_access_tok)



# Structring Dataset by extracting functions' features

### 1. Input-dependent loops

In [23]:
def for_loop_finder(function_node):
    fors = []
    for node in ast.walk(function_node):
        if isinstance(node, ast.For):
            fors.append(node)
    return fors

In [24]:
df = pd.DataFrame(columns = ['ID', 'number_of_for'])
for function_node in func_nodes_of_files:
    found_fors = for_loop_finder(function_node)
    
    ID = hashlib.sha1(str.encode(filename)).hexdigest()
    number_of_for = len(found_fors)
    
    new_row = pd.Series({"ID":ID, "number_of_for":number_of_for})
    df = pd.concat([df, new_row.to_frame().T], ignore_index=True)
df

,ID,number_of_for
0,567f964c581cc9644092b27d95f11eeb42833741,0
1,567f964c581cc9644092b27d95f11eeb42833741,0
2,567f964c581cc9644092b27d95f11eeb42833741,0
3,567f964c581cc9644092b27d95f11eeb42833741,0
4,567f964c581cc9644092b27d95f11eeb42833741,0
...,...,...
776,567f964c581cc9644092b27d95f11eeb42833741,0
777,567f964c581cc9644092b27d95f11eeb42833741,0
778,567f964c581cc9644092b27d95f11eeb42833741,0
779,567f964c581cc9644092b27d95f11eeb42833741,0


In [45]:
fors = [] #this should be dataframe instead
for function_node in func_nodes_of_files:
    found_fors = for_loop_finder(function_node)
    if len(found_fors) != 0:
        fors.extend(found_fors)
print(astor.to_source(for_nodes_of_files[0][0]))

for node in ([obj, target] + (related_nodes or [])):
    if isinstance(node, Project):
        create_timelines.post(node.node_id)

